In [42]:
import pandas as pd
import json

with open("index_data.json", "r", encoding="utf8") as f:
    data = json.load(f)

def get_unique_articles(data):
    seen_dois = set()
    unique_articles = []
    
    for d in data:
        for a in d['references']:
            doi = a['article_link']
            if doi not in seen_dois:
                seen_dois.add(doi)
                unique_articles.append({
                    'doi': doi,
                    'name': a['article_name']
                })
    
    return unique_articles

articles = get_unique_articles(data)

len(articles)

133

In [43]:
from pathlib import Path
import os

download_dir = Path(os.getcwd()).parent.parent / "data" / "papers"


In [44]:
import requests

import time
from urllib.parse import urljoin
import re

def download_from_scihub(doi, article_name, download_dir=download_dir, delay=2):
    """
    Скачивание одной статьи из Sci-Hub по DOI
    
    Args:
        doi: DOI статьи
        article_name: имя для сохранения файла
        download_dir: папка для сохранения
        delay: задержка перед запросом (секунды)
    """
    
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    
    # Актуальный домен Sci-Hub
    SCIHUB_BASE = "https://tesble.com/"
    
    # Заголовки чтобы имитировать браузер
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    session = requests.Session()
    session.headers.update(headers)
    
    try:
        print(f"Скачивание: {article_name}")
        print(f"DOI: {doi}")
        
        # Формируем URL для Sci-Hub
        scihub_url = urljoin(SCIHUB_BASE, doi)
        print(f"URL: {scihub_url}")
        
        # Получаем страницу Sci-Hub
        response = session.get(scihub_url, timeout=30)
        response.raise_for_status()
        
        # Ищем прямую ссылку на PDF в HTML
        pdf_url = extract_pdf_url(response.text)
        
        if not pdf_url:
            print(f"✗ Не найдена ссылка на PDF для {article_name}")
            return False, "PDF link not found"
        
        # Скачиваем PDF
        pdf_response = session.get(pdf_url, timeout=30)
        pdf_response.raise_for_status()
        
        # Проверяем что это действительно PDF
        content_type = pdf_response.headers.get('content-type', '')
        if 'application/pdf' not in content_type:
            print(f"✗ Получен не PDF файл для {article_name} (Content-Type: {content_type})")
            return False, "Not a PDF file"
        
        # Создаем безопасное имя файла
        safe_filename = create_safe_filename(article_name) + ".pdf"
        filepath = os.path.join(download_dir, safe_filename)
        
        # Сохраняем файл
        with open(filepath, 'wb') as f:
            f.write(pdf_response.content)
        
        print(f"✓ Успешно скачано: {safe_filename}")
        print(f"Размер файла: {len(pdf_response.content)} байт")
        return True, filepath
        
    except requests.exceptions.RequestException as e:
        print(f"✗ Ошибка сети для {article_name}: {str(e)}")
        return False, f"Network error: {str(e)}"
    except Exception as e:
        print(f"✗ Неожиданная ошибка для {article_name}: {str(e)}")
        return False, f"Unexpected error: {str(e)}"

def extract_pdf_url(html_content):
    """
    Извлекает URL PDF из HTML страницы Sci-Hub
    """
    # Паттерны для поиска ссылки на PDF
    patterns = [
        r'<embed[^>]*src="([^"]*\.pdf[^"]*)"',
        r'<iframe[^>]*src="([^"]*\.pdf[^"]*)"',
        r'<a[^>]*href="([^"]*\.pdf[^"]*)"[^>]*download',
        r'location\.href\s*=\s*["\']([^"\']*\.pdf[^"\']*)["\']'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, html_content, re.IGNORECASE)
        if match:
            pdf_url = match.group(1)
            # Если URL относительный, делаем его абсолютным
            if pdf_url.startswith('//'):
                return 'https:' + pdf_url
            elif pdf_url.startswith('/'):
                return 'https://tesble.com' + pdf_url
            else:
                return pdf_url
    
    return None

def create_safe_filename(filename):
    """
    Создает безопасное имя файла
    """
    # Заменяем недопустимые символы
    safe_name = re.sub(r'[\\/*?:"<>|]', '_', filename)
    safe_name = safe_name.replace('/', '_')
    safe_name = safe_name.replace(':', '_')
    # Убираем лишние пробелы
    safe_name = re.sub(r'\s+', ' ', safe_name).strip()
    # Ограничиваем длину имени
    if len(safe_name) > 100:
        safe_name = safe_name[:100]
    return safe_name

# Функция для скачивания списка статей
def download_articles_list(articles_data, download_dir="papers", delay_between=2):
    """
    Скачивание списка статей
    
    Args:
        articles_data: список словарей [{'doi': '...', 'name': '...'}, ...]
        download_dir: папка для сохранения
        delay_between: задержка между статьями
    """
    success_count = 0
    failed_count = 0
    results = []
    
    for i, article in enumerate(articles_data):
        doi = article.get('doi')
        name = article.get('name')
        
        if not doi or not name:
            print(f"✗ Пропущена статья {i+1}: отсутствует DOI или имя")
            failed_count += 1
            continue
        
        print(f"\n[{i+1}/{len(articles_data)}] ", end="")
        success, result = download_from_scihub(doi, name, download_dir, delay=0)
        
        results.append({
            'article_name': name,
            'doi': doi,
            'success': success,
            'result': result
        })
        
        if success:
            success_count += 1
        else:
            failed_count += 1
        
        # Задержка между статьями (кроме последней)
        if i < len(articles_data) - 1:
            time.sleep(delay_between)
    
    print(f"\n=== ИТОГИ ===")
    print(f"Успешно: {success_count}, Неудачно: {failed_count}")
    return results

# Пример использования
if __name__ == "__main__":
    # Пример для одной статьи
    doi = "10.1007/bf00031911"
    article_name = "Important Research Paper 2024"
    
    success, result = download_from_scihub(doi, article_name)
    
    if success:
        print(f"Статья сохранена как: {result}")
    else:
        print(f"Ошибка: {result}")
    
    # Пример для списка статей
    # articles = [
    #     {'doi': '10.1007/bf00031911', 'name': 'Biology Research Paper'},
    #     {'doi': '10.1038/s41586-021-03375-9', 'name': 'Nature Important Study'},
    #     {'doi': '10.1126/science.abe4473', 'name': 'Science Journal Article'}
    # ]
    
    results = download_articles_list(articles)

Скачивание: Important Research Paper 2024
DOI: 10.1007/bf00031911
URL: https://tesble.com/10.1007/bf00031911
✓ Успешно скачано: Important Research Paper 2024.pdf
Размер файла: 389090 байт
Статья сохранена как: c:\Users\Admin\Desktop\pythonProjects\IndexTable\data\papers\Important Research Paper 2024.pdf

[1/133] Скачивание: Gitelson, Anatoly A.; Merzlyak, Mark N.; Lichtenthaler, Hartmut K. - Detection of Red Edge Position and Chlorophyll Content by Reflectance Measurements Near 700 nm

DOI: 10.1016/s0176-1617(96)80285-9
URL: https://tesble.com/10.1016/s0176-1617(96)80285-9
✓ Успешно скачано: Gitelson, Anatoly A.; Merzlyak, Mark N.; Lichtenthaler, Hartmut K. - Detection of Red Edge Position .pdf
Размер файла: 958591 байт

[2/133] Скачивание: le Maire, G.; Francois, C.; Dufrene, E. - Towards universal broad leaf chlorophyll indices using PROSPECT simulated database and hyperspectral reflectance measurements

DOI: 10.1016/j.rse.2003.09.004
URL: https://tesble.com/10.1016/j.rse.2003.09.004